### Imports

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d.axes3d import Axes3D
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info.states import Statevector
from time import sleep

## Config

### Inline vs rotatable views
If you want to be able to rorate the figures, run the following command.
If you just want to have them inline, do not run it.

In [26]:
%matplotlib qt
# %matplotlib inline

In [27]:
from ipynb.fs.full.ansatz import gen_expressive_ansatz_1qubit
from mub_state_gen import generate_mub_state_circ
from experiment_utils import run_and_record_landscape, run_and_record_vqe_expressive_1q
from landscape import TotalLandscapeResult, calculate_energy_landscape

In [28]:
def sphere_to_cart(theta, phi) -> tuple[np.float64, np.float64, np.float64]:
    x = np.cos(theta) * np.sin(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(phi)
    return x,y,z

### Draw regular sphere + line Axes

In [29]:
def draw_bloch():
    thetas = np.linspace(0, 2*np.pi, 100)
    phis = np.linspace(0, np.pi, 100)

    x = np.outer(np.cos(thetas), np.sin(phis))
    y = np.outer(np.sin(thetas), np.sin(phis))
    z = np.outer(np.ones(np.size(thetas)), np.cos(phis))

    # coords = [sphere_to_cart(*point) for point in zip(thetas, phis)]
    # xs, ys, zs = (np.array(l) for l in zip(*coords))
    # print(type(xs))
    # ax.plot_surface(xs, ys, zs)

    ax = plt.axes(projection="3d")
    plt.grid(False)
    plt.axis('off')
    
    # # this is used to set the graph color to blue
    # blue = np.array([0., 0., 1.])
    # rgb = np.tile(blue, (z.shape[0], z.shape[1], 1))
    
    ls = LightSource()
    illuminated_surface = ls.shade(z, cmap=plt.get_cmap("Blues"))
    
    ax.plot_surface(x,y,z, alpha=0.15, linewidth=0, antialiased=True, facecolors=illuminated_surface)
    # ax.plot_surface(x,y,z, alpha=0.15, linewidth=0, antialiased=True, color="Grey")

    L=1.5

    # Axes
    ax.set_xlim(-L,L)
    ax.set_ylim(-L,L)
    ax.set_zlim(-L,L)
    # ax.set_ylabel('Y axis')
    # ax.set_zlabel('Z axis')

    ax.quiver(-1.5,0,0, 3,0,0, arrow_length_ratio=0.1)
    ax.quiver(0,-1.5,0, 0,3,0, arrow_length_ratio=0.1)
    ax.quiver(0,0,-1.5, 0,0,3, arrow_length_ratio=0.1)
    
    ax.text(1.5,0,0,'X axis')
    ax.text(0,1.5,0,'Y axis')
    ax.text(0,0,1.5,'Z axis')
    
    
    # Reference Points
    ax.text(0,0,1,r"$|0\rangle$")
    ax.text(0,0,-1,r"$|1\rangle$")
    ax.text(1,0,0,r"$|+\rangle$")
    ax.text(-1,0,0,r"$|-\rangle$")
    ax.text(0,1,0,r"$|+_{i}\rangle$")
    ax.text(0,-1,0,r"$|-_{i}\rangle$")
    return ax
    # plt.show()


### From State to Bloch Sphere polar coords

In [30]:
def vec_to_sphericals(vec: np.array) -> tuple[np.float64, np.float64]:
    # vec = state.data
    alpha, beta = vec[0], vec[1]
    real_alpha = np.abs(alpha)
    phi = 2 * np.arccos(real_alpha)
    theta = np.angle(beta) - np.angle(alpha)
    return theta, phi
    
def qubit_to_sphericals(state: Statevector) -> tuple[np.float64, np.float64]:
    return vec_to_sphericals(state.data)

### draw onto the sphere

In [68]:
def add_to_sphere(ax: Axes3D, theta, phi, color="Blue", size=10):
    x,y,z = sphere_to_cart(theta, phi)
    ax.scatter3D([x], [y], [z], color=color, s=size)

### Example - MUB state gen. and drawing

In [69]:
state_0 = Statevector.from_instruction(generate_mub_state_circ(0,0,1,[0]))
ax = draw_bloch()
add_to_sphere(ax, *qubit_to_sphericals(state_0))
state_plusi = Statevector.from_instruction(generate_mub_state_circ(0,2,1,[0]))
add_to_sphere(ax, *qubit_to_sphericals(state_plusi), color="RED")


## Landscaping

In [70]:
def value_to_color(val: np.float64):
    return "Blue"

def draw_MUB_values(ax: Axes3D, res: TotalLandscapeResult):
    for mub_res in res.mub_results:
        for state_res in mub_res[0]:
            color = value_to_color(state_res.value)
            state = Statevector.from_instruction(state_res.state_circuit)
            add_to_sphere(ax, *qubit_to_sphericals(state))
            


## VQE Visualization

In [71]:
def ansatz_params_to_sphericals(ansatz_params: list[np.ndarray]) -> list[tuple[np.float64, np.float64]]:
    sphericals = []
    for iter_params in ansatz_params:
        circ = gen_expressive_ansatz_1qubit()
        circ.assign_parameters(iter_params, inplace=True)
        state = Statevector.from_instruction(circ)
        sphericals.append(qubit_to_sphericals(state))
    return sphericals

### Auxiliary: locate optimal value

In [73]:
def find_ground_state(ham: SparsePauliOp):
    vals, vecs = np.linalg.eig(ham.to_matrix())
    min_val_idx = vals.argmin()
    ground_state = vecs[:,min_val_idx]
    if ground_state[0].real < 0:
        ground_state *= -1
    return ground_state

def draw_optimal_point(ham: SparsePauliOp, ax: Axes3D):
    ground_state = find_ground_state(ham)
    spherical = vec_to_sphericals(ground_state)
    add_to_sphere(ax, *spherical, color='red', size=50)

## All together, now!

In [74]:
ham = SparsePauliOp(data= ['I', 'X', 'Y', 'Z'], coeffs= [0, 0.5, 0, 0.5])
results = run_and_record_landscape(ham, 1)
draw_MUB_values(ax, results)
vqe_results = run_and_record_vqe_expressive_1q(results)
state_names = [r"$|0\rangle$", r"$|1\rangle$", r"$|+\rangle$", r"$|-\rangle$", r"$|+_{i}\rangle$", r"$|-_{i}\rangle$"]
def foo(vqe_result, state_name):
    ax = draw_bloch()
    # add target point
    draw_optimal_point(ham, ax)
    thetas_as_sphericals = ansatz_params_to_sphericals(vqe_result.thetas_list)
    thetas_as_carts = [sphere_to_cart(*t) for t in thetas_as_sphericals]
    xs, ys, zs = list(zip(*thetas_as_carts))
    ax.plot3D(xs, ys, zs, alpha=0.3)
    ax.scatter3D(xs, ys, zs, c=list(range(len(xs))), cmap='inferno')
    ax.set_title(f"run from state {state_name}")
    plt.show()

# for vqe_result, state_name in zip(vqe_results, state_names):

attempting all MUB states over the operator SparsePauliOp(['I', 'X', 'Y', 'Z'],
              coeffs=[0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j])
Energy Landscape:
Energy Histogram:
The operator SparsePauliOp(['I', 'X', 'Y', 'Z'],
              coeffs=[0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j]) has the exact value -0.7071067811865475.
Now trying to reach the value from different MUB points.
running from state of index subset 0, MUB 0, state 0 and value 0.5
10: -0.49652232221547965
20: -0.7068828443551921
30: -0.7071023684707732
40: -0.707106779715569
50: -0.707106780980516
running from state of index subset 0, MUB 0, state 1 and value -0.5
10: -0.686511268820321
20: -0.7070909335708759
30: -0.7071059289106745
40: -0.7071067044800836
50: -0.7071067793603923
running from state of index subset 0, MUB 1, state 0 and value 0.5
10: -0.7044153001313705
20: -0.7053105341000518
30: -0.7070020692443535
40: -0.70710651009581
50: -0.707106777569363
running from state of index subset 0, MUB 1, state 1 and value

In [75]:
foo(vqe_results[0], state_names[0])

In [66]:
foo(vqe_results[1], state_names[1])

In [50]:
foo(vqe_results[2], state_names[2])